In [1]:
import re
import string
import pandas as pd
from IPython.display import display
from sklearn.preprocessing import LabelEncoder

# For machine learning
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
import numpy as np


# To ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv(r"C:\Users\ASUS\Desktop\Playground\Data Cleaning Test Folder\Dataset\new_data.csv")

In [5]:
df.head()

,Experience,Qualifications,Salary Range,location,Country,Work Type,Job Posting Date,Contact Person,Job Title,Role,...,skills,Responsibilities,Company,Company Profile,exp_min,exp_max,exp_avg,sal_min,sal_max,sal_avg
0,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,Intern,2022-04-24,Brandon Cunningham,Digital Marketing Specialist,Social Media Manager,...,"Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie...",5.0,15.0,10.0,59000.0,99000.0,79000.0
1,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,Intern,2022-12-19,Francisco Larsen,Web Developer,Frontend Web Developer,...,"HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com...",2.0,12.0,7.0,56000.0,116000.0,86000.0
2,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",Temporary,2022-09-14,Gary Gibson,Operations Manager,Quality Control Manager,...,Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P...",0.0,12.0,6.0,61000.0,104000.0,82500.0
3,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,Full-Time,2023-02-25,Joy Lucero,Network Engineer,Wireless Network Engineer,...,Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O...",4.0,11.0,7.5,65000.0,91000.0,78000.0
4,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,Intern,2022-10-11,Julie Johnson,Event Manager,Conference Manager,...,Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ...",1.0,12.0,6.5,64000.0,87000.0,75500.0


In [6]:
# Categorical columns to encode
categorical_columns = ['Work Type', 'location', 'Country', 
                       'Qualifications', 'Role', 'Company', 'Job Title']

# Initializing and applying Label Encoding
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le  # Save encoder for later use

# Confirm encoding
df.head()

,Experience,Qualifications,Salary Range,location,Country,Work Type,Job Posting Date,Contact Person,Job Title,Role,...,skills,Responsibilities,Company,Company Profile,exp_min,exp_max,exp_avg,sal_min,sal_max,sal_avg
0,5 to 15 Years,6,$59K-$99K,59,92,2,2022-04-24,Brandon Cunningham,31,312,...,"Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",425,"{""Sector"":""Diversified"",""Industry"":""Diversifie...",5.0,15.0,10.0,59000.0,99000.0,79000.0
1,2 to 12 Years,4,$56K-$116K,11,198,2,2022-12-19,Francisco Larsen,145,147,...,"HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",617,"{""Sector"":""Financial Services"",""Industry"":""Com...",2.0,12.0,7.0,56000.0,116000.0,86000.0
2,0 to 12 Years,9,$61K-$104K,102,114,4,2022-09-14,Gary Gibson,90,273,...,Quality control processes and methodologies St...,Establish and enforce quality control standard...,816,"{""Sector"":""Insurance"",""Industry"":""Insurance: P...",0.0,12.0,6.0,61000.0,104000.0,82500.0
3,4 to 11 Years,9,$65K-$91K,152,20,1,2023-02-25,Joy Lucero,83,375,...,Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",402,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O...",4.0,11.0,7.5,65000.0,91000.0,78000.0
4,1 to 12 Years,7,$64K-$87K,172,39,2,2022-10-11,Julie Johnson,38,60,...,Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,174,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ...",1.0,12.0,6.5,64000.0,87000.0,75500.0


In [7]:
# print("Label mapping:", dict(zip(le.classes_, range(len(le.classes_)))))

In [8]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'\n', ' ', text)  # Remove newlines
    text = re.sub(r'\[.*?\]', '', text)  # Remove text in brackets
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)  # Remove punctuation
    text = re.sub(r'\w*\d\w*', '', text)  # Remove words with numbers
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text.strip()  # Remove leading/trailing spaces

# Apply cleaning to text columns
text_columns = ['Job Description', 'skills', 'Responsibilities', 'Benefits', 'Company Profile']
for col in text_columns:
    df[col] = df[col].apply(clean_text)

# Check the cleaned data
df[text_columns].head()

,Job Description,skills,Responsibilities,Benefits,Company Profile
0,social media managers oversee an organizations...,social media platforms eg facebook twitter ins...,manage and grow social media accounts create e...,flexible spending accounts fsas relocation ass...,sectordiversifiedindustrydiversified financial...
1,frontend web developers design and implement u...,html css javascript frontend frameworks eg rea...,design and code user interfaces for websites e...,health insurance retirement plans paid time of...,sectorfinancial servicesindustrycommercial s d...
2,quality control managers establish and enforce...,quality control processes and methodologies st...,establish and enforce quality control standard...,legal assistance bonuses and incentive program...,sectorinsuranceindustryinsurance property and ...
3,wireless network engineers design implement an...,wireless network design and architecture wifi ...,design configure and optimize wireless network...,transportation benefits professional developme...,sectorenergyindustrymining crudeoil production...
4,a conference manager coordinates and manages c...,event planning conference logistics budget man...,specialize in conference and convention planni...,flexible spending accounts fsas relocation ass...,sectorenergyindustryenergy oil gas exploration...


In [9]:
# Combine all relevant text columns into one feature
df['combined_text'] = (
    df['Job Title'].astype(str) + " " +  # Prioritize job title
    df['Job Description'] + " " +
    df['skills'] + " " +
    df['Responsibilities'] + " " +
    df['Benefits'] + " " +
    df['Company Profile']
)

# Confirm the combined text
df['combined_text'].head()

0    31 social media managers oversee an organizati...
1    145 frontend web developers design and impleme...
2    90 quality control managers establish and enfo...
3    83 wireless network engineers design implement...
4    38 a conference manager coordinates and manage...
Name: combined_text, dtype: object

In [10]:
df.head()

,Experience,Qualifications,Salary Range,location,Country,Work Type,Job Posting Date,Contact Person,Job Title,Role,...,Responsibilities,Company,Company Profile,exp_min,exp_max,exp_avg,sal_min,sal_max,sal_avg,combined_text
0,5 to 15 Years,6,$59K-$99K,59,92,2,2022-04-24,Brandon Cunningham,31,312,...,manage and grow social media accounts create e...,425,sectordiversifiedindustrydiversified financial...,5.0,15.0,10.0,59000.0,99000.0,79000.0,31 social media managers oversee an organizati...
1,2 to 12 Years,4,$56K-$116K,11,198,2,2022-12-19,Francisco Larsen,145,147,...,design and code user interfaces for websites e...,617,sectorfinancial servicesindustrycommercial s d...,2.0,12.0,7.0,56000.0,116000.0,86000.0,145 frontend web developers design and impleme...
2,0 to 12 Years,9,$61K-$104K,102,114,4,2022-09-14,Gary Gibson,90,273,...,establish and enforce quality control standard...,816,sectorinsuranceindustryinsurance property and ...,0.0,12.0,6.0,61000.0,104000.0,82500.0,90 quality control managers establish and enfo...
3,4 to 11 Years,9,$65K-$91K,152,20,1,2023-02-25,Joy Lucero,83,375,...,design configure and optimize wireless network...,402,sectorenergyindustrymining crudeoil production...,4.0,11.0,7.5,65000.0,91000.0,78000.0,83 wireless network engineers design implement...
4,1 to 12 Years,7,$64K-$87K,172,39,2,2022-10-11,Julie Johnson,38,60,...,specialize in conference and convention planni...,174,sectorenergyindustryenergy oil gas exploration...,1.0,12.0,6.5,64000.0,87000.0,75500.0,38 a conference manager coordinates and manage...


In [11]:
text_columns_to_drop = ["Experience","Salary Range","Job Posting Date","Job Portal", "Contact Person", "Job Description", "Benefits", "skills", "Responsibilities", "Company Profile"]
df = df.drop(columns=text_columns_to_drop)

In [12]:
df.head()

,Qualifications,location,Country,Work Type,Job Title,Role,Company,exp_min,exp_max,exp_avg,sal_min,sal_max,sal_avg,combined_text
0,6,59,92,2,31,312,425,5.0,15.0,10.0,59000.0,99000.0,79000.0,31 social media managers oversee an organizati...
1,4,11,198,2,145,147,617,2.0,12.0,7.0,56000.0,116000.0,86000.0,145 frontend web developers design and impleme...
2,9,102,114,4,90,273,816,0.0,12.0,6.0,61000.0,104000.0,82500.0,90 quality control managers establish and enfo...
3,9,152,20,1,83,375,402,4.0,11.0,7.5,65000.0,91000.0,78000.0,83 wireless network engineers design implement...
4,7,172,39,2,38,60,174,1.0,12.0,6.5,64000.0,87000.0,75500.0,38 a conference manager coordinates and manage...


In [15]:
# Stored the processed 1.6 million rows data
# df.to_csv("Processed_Job_Data.csv",index=False)

# Reduced the df size to 2 Lac rows because of system specs limitation
# req_df = df.head(200000)

In [16]:
from scipy.sparse import hstack

tfidf = TfidfVectorizer(
    max_features=1000,          # Adjust based on RAM
    stop_words='english',
    ngram_range=(1, 2)         # Bigrams if needed
)

X_text = tfidf.fit_transform(req_df['combined_text'])

X_numeric = req_df[['Qualifications', 'location', 'Country', 'Work Type', 'Role', 'Company', 'exp_min', 'exp_max', 'exp_avg', 'sal_min', 'sal_max', 'sal_avg']].values  # Example numeric cols
X_final = hstack([X_numeric, X_text])
y = req_df['Job Title'].values 

In [17]:
# If you used hstack([X_numeric, X_text])
print("Shape before:", X_numeric.shape, X_text.shape)
print("Shape after:", X_final.shape)  # If columns increased unexpectedly, the target may be included

Shape before: (200000, 12) (200000, 1000)
Shape after: (200000, 1012)


In [20]:
# 2. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42
)

# 3. Pipeline Options
pipeline_with_svd = Pipeline([
    ('svd', TruncatedSVD(n_components=300)),  # Dimensionality reduction
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1  # Use all CPU cores
    ))
])

pipeline_without_svd = Pipeline([
    ('scaler', StandardScaler(with_mean=False)),
    ('model', RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ))
])

# 4. Training & Evaluation
print("Training with SVD:")
%time pipeline_with_svd.fit(X_train, y_train)
print(f"Test Accuracy: {pipeline_with_svd.score(X_test, y_test):.4f}")

print("\nTraining without SVD:")
%time pipeline_without_svd.fit(X_train, y_train)
print(f"Test Accuracy: {pipeline_without_svd.score(X_test, y_test):.4f}")

Training with SVD:
CPU times: total: 25min 39s
Wall time: 4min 31s
Test Accuracy: 0.9253

Training without SVD:
CPU times: total: 1min 48s
Wall time: 18.1 s
Test Accuracy: 0.8084


In [23]:
# from sklearn.metrics import classification_report, confusion_matrix

# y_pred_svd = pipeline_with_svd.predict(X_test)
# print("=== With SVD ===")
# print(f"Accuracy: {pipeline_with_svd.score(X_test, y_test):.4f}")
# print(classification_report(y_test, y_pred_svd))

# # For Non-SVD Pipeline (fixed version)
# y_pred_no_svd = pipeline_without_svd.predict(X_test)
# print("\n=== Without SVD ===")
# print(f"Accuracy: {pipeline_without_svd.score(X_test, y_test):.4f}")
# print(classification_report(y_test, y_pred_no_svd))

In [19]:
from joblib import dump

# Save both pipelines
dump(pipeline_with_svd, 'pipeline_with_svd.joblib')
# dump(pipeline_without_svd, 'pipeline_without_svd.joblib')

['pipeline_with_svd.joblib']

#### For identifying the labels that are associated to each element or tuple in the data:

In [61]:
# # For a specific column (e.g., 'Qualifications')
# print("Qualifications Mapping:")
# for original, encoded in zip(label_encoders['Qualifications'].classes_, 
#                             range(len(label_encoders['Qualifications'].classes_))):
#     print(f"{original} → {encoded}")

In [59]:
# print("location Mapping:")
# for original, encoded in zip(label_encoders['location'].classes_, 
#                             range(len(label_encoders['location'].classes_))):
#     print(f"{original} → {encoded}")

In [60]:
# print("Country Mapping:")
# for original, encoded in zip(label_encoders['Country'].classes_, 
#                             range(len(label_encoders['Country'].classes_))):
#     print(f"{original} → {encoded}")

In [58]:
# print("Work Type Mapping:")
# for original, encoded in zip(label_encoders['Work Type'].classes_, 
#                             range(len(label_encoders['Work Type'].classes_))):
#     print(f"{original} → {encoded}")

In [57]:
# print("Job Title Mapping:")
# for original, encoded in zip(label_encoders['Job Title'].classes_, 
#                             range(len(label_encoders['Job Title'].classes_))):
#     print(f"{original} → {encoded}")

In [56]:
# print("Role Mapping:")
# for original, encoded in zip(label_encoders['Role'].classes_, 
#                             range(len(label_encoders['Role'].classes_))):
#     print(f"{original} → {encoded}")

In [55]:
# print("Company Mapping:")
# for original, encoded in zip(label_encoders['Company'].classes_, 
#                             range(len(label_encoders['Company'].classes_))):
#     print(f"{original} → {encoded}")

#### Storing the mapped labels in a JSON file:

In [35]:
label_mappings = {
    'Qualifications': dict(zip(label_encoders['Qualifications'].classes_,
                               range(len(label_encoders['Qualifications'].classes_)))),
    'location': dict(zip(label_encoders['location'].classes_,
                         range(len(label_encoders['location'].classes_)))),
    'Country': dict(zip(label_encoders['Country'].classes_,
                        range(len(label_encoders['Country'].classes_)))),
    'Work Type': dict(zip(label_encoders['Work Type'].classes_,
                          range(len(label_encoders['Work Type'].classes_)))),
    'Job Title': dict(zip(label_encoders['Job Title'].classes_,
                          range(len(label_encoders['Job Title'].classes_)))),
    'Role': dict(zip(label_encoders['Role'].classes_,
                     range(len(label_encoders['Role'].classes_))))
}

# Save as JSON
import json
with open('label_mappings.json', 'w', encoding='utf-8') as f:
    json.dump(label_mappings, f, indent=2, ensure_ascii=False)

# Save as a Joblib file
from joblib import dump
dump(label_mappings, 'label_mappings.joblib')

['label_mappings.joblib']

In [44]:
from joblib import dump

# Assuming you have your tfidf vectorizer from training
dump(tfidf, 'tfidf_vectorizer.joblib') 
print("TF-IDF vectorizer saved!")

TF-IDF vectorizer saved!


#### Testing how well the model generalizes:

In [54]:
import os
from scipy.sparse import csr_matrix
def predict_job(input_data, pipeline_path='pipeline_with_svd.joblib', 
               mappings_path='label_mappings.json'):
    try:
        # Load assets
        pipeline = load(pipeline_path)
        with open(mappings_path) as f:
            mappings = json.load(f)
        
        # --- Create properly dimensioned input ---
        # 1. Prepare numeric features (7 features)
        numeric_features = np.array([
            mappings['Qualifications'][input_data['Qualifications']],
            mappings['location'][input_data['location']],
            mappings['Country'][input_data['Country']],
            mappings['Work Type'][input_data['Work Type']],
            mappings['Role'][input_data['Role']],
            input_data.get('exp_min', 0),
            input_data.get('sal_avg', 50000)
        ]).reshape(1, -1)
        
        # 2. Prepare text features (1000 features)
        if 'tfidf_vectorizer.joblib' in os.listdir():
            tfidf = load('tfidf_vectorizer.joblib')
            text_features = tfidf.transform([input_data['combined_text']])
        else:
            # Create dummy text features if vectorizer missing
            text_features = csr_matrix((1, 1000))  # 1000 zero features
        
        # 3. Combine with correct dimensions (1007 total)
        X = hstack([numeric_features, text_features])
        
        # 4. Pad with zeros if needed (for 1012 expected features)
        if X.shape[1] < 1012:
            padding = csr_matrix((1, 1012 - X.shape[1]))
            X = hstack([X, padding])
        
        # Predict
        pred_encoded = pipeline.predict(X)[0]
        proba = pipeline.predict_proba(X)[0]
        
        return {
            'predicted_job': next(k for k,v in mappings['Job Title'].items() if v == pred_encoded),
            'confidence': float(np.max(proba)),
            'feature_count': X.shape[1]  # Should show 1012
        }
    
    except Exception as e:
        return {"error": str(e)}


print(predict_job({
    "Qualifications": "B.Tech",
    "location": "New Delhi",
    "Country": "India",
    "Work Type": "Full-Time",
    "Role": "API Developer",
    "combined_text": "Technology",  # Will be ignored if no TF-IDF
    "exp_min": 2
}))

{'predicted_job': 'UX/UI Designer', 'confidence': 0.19, 'feature_count': 1012}
